In [ ]:
from queue import Queue
from threading import Thread
import sys
import time
import cv2

class VideoReader:
    def __init__(self, video_file, queue_size=128):
        # initialize the file video stream along with the boolean
        # used to indicate if the thread should be stopped or not
        self.stream = cv2.VideoCapture(video_file)
        self.stopped = False
        # initialize the queue used to store frames read from
        # the video file
        self.Q = Queue(maxsize=queue_size)
        self.frame_num = 0
        self.false_count = 0 
        print('reading video from ', video_file)
        

    def start(self):
        # start a thread to read frames from the file video stream
        t = Thread(target=self.update, args=())
        t.daemon = True
        t.start()
        time.sleep(5)
        print('starting video reader...')
        return self

    def update(self):
        # keep looping infinitely
#         frame_num = 0
        print('update starting...')
        while True:
#             if frame_num == 100:
#                 break
            # if the thread indicator variable is set, stop the
            # thread
            if self.stopped:
                return

            # otherwise, ensure the queue has room in it
            if not self.Q.full():
                # read the next frame from the file

                (grabbed, frame) = self.stream.read()
                self.frame_num += 1
                
                
                # add the frame to the queue
                if not grabbed:
                    if self.false_count > 500:
                        print('no more frames')
                        self.stop()
                        return
                    else:
                        if self.frame_num % 25 == 0:
                            print('queue, {}, {}'.format(self.frame_num, grabbed))
                        self.false_count += 1
                else:
                    self.Q.put(frame)
                    self.false_count = 0
                

    def read(self):
        # return next frame in the queue
        return self.Q.get()


    def more(self):
        # return True if there are still frames in the queue
        return self.Q.qsize() > 0

    def stop(self):
        # indicate that the thread should be stopped
        self.stopped = True
        self.stream.release()
        print('stopped')
    
    def is_stopped(self):
        return self.stopped
    
    def is_full(self):
        return self.Q.full()
        
        
    def close(self):
        self.stop()

In [ ]:
import glob
import os
parent_video_folder = '.../kasanka-bats/gopros/' 

all_video_files = []
for date in ['16Nov', '17Nov', '18Nov', '19Nov', '20Nov']:
    all_video_files.extend(glob.glob(parent_video_folder + date + '/*/*.MP4'))
#     video_files.extend(glob.glob(parent_video_folder + date + '/BBC/*.MP4'))
all_video_files.sort()
    # vf = glob.glob(parent_video_folder + '/14_11_g/*.MP4')
    # vf.sort()
    # video_files.extend(vf)
# print(*video_files, sep='\n')

median_files = 0
mean_files = 0
max_files = 0

video_files = []

for video_file in all_video_files:
    
    video_name = os.path.basename(video_file)
    video_name = video_name.split('.')[-2]
    parent_output_folder = '.../kasanka-bats/processed/deep-learning'
    observation_name = video_file.split('/')[-2]
    output_folder = os.path.join(parent_output_folder, video_file.split('/')[-3], observation_name)
    
    if os.path.exists(os.path.join(output_folder, f'mean-blue-{video_name}.npy')):
        mean_files += 1
    else:
        video_files.append(video_file)
    if os.path.exists(os.path.join(output_folder, f'median-blue-{video_name}.npy')):
        median_files += 1
    if os.path.exists(os.path.join(output_folder, f'max-blue-{video_name}.npy')):
        max_files += 1

print(f'num mean: {mean_files}, num median: {mean_files}, num max: {max_files}')
print(f'fraction mean: {mean_files / lenall_(video_files)}')

In [ ]:
import os
from multiprocessing import Pool
import numpy as np

import time
skip_existing_folders = False
process_every_n_frames = 1



# existing_image_folders = glob.glob(output_folder + '/*')
# existing_folder_names = [os.path.basename(image_folder) for image_folder in existing_image_folders]
        


    
def get_darkness_info(video_file):
    video_stream = None
    try:
        parent_output_folder = '.../kasanka-bats/frames/'
        parent_output_folder = '.../bats'
        parent_output_folder = '.../kasanka-bats/processed/deep-learning'

        observation_name = video_file.split('/')[-2]

        output_folder = os.path.join(parent_output_folder, video_file.split('/')[-3], observation_name)
        
        os.makedirs(output_folder, exist_ok=True)

        video_name = os.path.basename(video_file)
        video_name = video_name.split('.')[-2]

        print('Processing', video_name)

        frame_count = 0
        video_stream = VideoReader(video_file).start()
        time.sleep(1)
        last_frame = None

        mean_blue = []
        median_blue = []
        max_blue = []

        started = False

        while video_stream.more():

            if video_stream.is_full() or video_stream.is_stopped():
                if not started:
                    tic = time.time()
                    started=True
                if frame_count % 4000 == 0:
                    print(frame_count, 'frames processed')

                frame = video_stream.read()
                blue_frame = frame[..., 0] # 0 because BGR from opencv

                mean_blue.append(np.mean(blue_frame))
                median_blue.append(np.median(blue_frame))
                max_blue.append(blue_frame.max())

                frame_count += 1

        video_stream.stop()

        np.save(os.path.join(output_folder, f'mean-blue-{video_name}.npy'), np.array(mean_blue))
        np.save(os.path.join(output_folder, f'median-blue-{video_name}.npy'), np.array(median_blue))
        np.save(os.path.join(output_folder, f'max-blue-{video_name}.npy'), np.array(max_blue))

        toc = time.time()
        print(f'total_time {toc - tic}, fps: {frame_count / (toc-tic)}')
        
    except:
        if video_stream:
            video_stream.stop()
            print(f'FAILED: {video_file}')
    

if __name__ == '__main__':


    with Pool(processes=4) as pool:
        pool.map(get_darkness_info, video_files)

In [ ]:
video_file = video_files[0]
parent_output_folder = '.../kasanka-bats/processed/deep-learning'
    
observation_name = video_file.split('/')[-2]

output_folder = os.path.join(parent_output_folder, video_file.split('/')[-3], observation_name)
print(output_folder)

In [ ]:
mean_files = sorted(glob.glob(os.path.join(output_folder, 'mean-blue-*.npy')))
means = []
for file in mean_files:
    means.append(np.load(file))
means = np.hstack(means)
print(means.shape)

In [ ]:
camera = 'BBC'
date = '16Nov'

images_folder = f'.../Elements/bats/{date}/{camera}'
image_folder = os.path.join(images_folder, '*', '*.jpg')
image_files = sorted(glob.glob(image_folder))
print(image_folder)
print(f'{len(image_files)} files found')

In [ ]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(20,20))
# plt.plot(means, 'o', )

# interval = 1000

# for file_ind, file in enumerate(image_files[::interval]):
#     frame = plt.imread(file)
#     mean = np.mean(frame[..., 2])
#     plt.scatter(file_ind*interval, mean, c='r', s=100)

In [ ]:
import glob

In [ ]:
parent_output_folder = '.../kasanka-bats/processed/deep-learning'
day_folders = sorted(glob.glob(os.path.join(parent_output_folder, '*')))
for day_folder in day_folders[:]:
    camera_folders = sorted(glob.glob(os.path.join(day_folder, '*')))
    for camera_folder in camera_folders[:]:
        if os.path.exists(os.path.join(camera_folder, 'blue-means.npy')):
            print('already seen', camera_folder, day_folder)
            continue
        mean_files = sorted(glob.glob(os.path.join(camera_folder, 'mean-*.npy')))
        means = []
        for f in mean_files:
            m = np.load(f)
            means.append(m)
        means = np.hstack(means)
        np.save(os.path.join(camera_folder, 'blue-means.npy'), means)
        for f in mean_files:
            os.remove(f)
        
        max_files = sorted(glob.glob(os.path.join(camera_folder, 'max-*.npy')))
        maxs = []
        for f in max_files:
            m = np.load(f)
            maxs.append(m)
        maxs = np.hstack(maxs)
        np.save(os.path.join(camera_folder, 'blue-maxs.npy'), maxs)
        for f in max_files:
            os.remove(f)
        
        median_files = sorted(glob.glob(os.path.join(camera_folder, 'median-*.npy')))
        medians = []
        for f in median_files:
            m = np.load(f)
            medians.append(m)
        medians = np.hstack(medians)
        np.save(os.path.join(camera_folder, 'blue-medians.npy'), medians)
        for f in median_files:
            os.remove(f)

In [ ]:
blue_means = np.load(os.path.join(camera_folder, 'blue-maxs.npy'))

In [ ]:
plt.plot(blue_means)